In [ ]:
import numpy as np
import pandas as pd
import sys

data='/content/102116110-data.csv'
def read_data_for_topsis(data):
    try:
        dataset = pd.read_csv(data)
        return dataset
    except FileNotFoundError:
        print("Error: File not Found")
        return None


def weights_and_impacts_for_topsis(weights, impacts):
    try:
        weights_array_for_topsis = np.fromiter( map(float, weights.split(',')), dtype=float)
        used_symbols = impacts.split(',')
        symbol_to_number_for_topsis= {'+': 1, '-': -1}
        num = [symbol_to_number_for_topsis[symbol.strip()] for symbol in used_symbols]
        impacts_array_for_topsis = np.array(num)
        return weights_array_for_topsis, impacts_array_for_topsis
    except ValueError as ed:
        print(f"Error parsing weights and impacts: {ed}")
        return None, None


def check_columns_for_topsis(dataset):
    try:
        return len(dataset.iloc[0]) >= 3
    except Exception as ed:
        print(f"Error checking for  columns: {ed}")
        return False


def normalize_data_for_topsis(dataset):
    num_values = dataset.iloc[:, 1:].values.tolist()
    num_values = np.array(num_values)
    normalized_matrix_ = num_values / \
np.sqrt((num_values ** 2).sum(axis=0))
    return normalized_matrix_


def do_topsis(normalized_data_for_topsis, weights, impacts):
    weighted_matrix_ = normalized_data_for_topsis * weights * impacts

    positive_ideal_solution = np.abs(np.array([weighted_matrix_[:, i].max() if weights[i] > 0 else weighted_matrix_[:, i].min() for i in range(len(weights))]))
    negative_ideal_solution = np.abs(np.array([weighted_matrix_[:, i].min(
 ) if weights[i] > 0 else weighted_matrix_[:, i].max() for i in range(len(weights))]))

    weighted_matrix_ = np.abs(weighted_matrix_)

    euclidean_dist_positive_ideal = np.sqrt(((weighted_matrix_ - positive_ideal_solution) ** 2).sum(axis=1))
    euclidean_dist_negative_ideal = np.sqrt(((weighted_matrix_ - negative_ideal_solution) ** 2).sum(axis=1))

    closeness_coeff = euclidean_dist_negative_ideal / \
        (euclidean_dist_positive_ideal + euclidean_dist_negative_ideal)
    return closeness_coeff


def check_num_values(dataset):
    try:
        numeric_columns = dataset.iloc[:, 1:].apply( pd.to_numeric, errors='coerce').notnull().all(axis=0)
        return numeric_columns.all()
    except Exception as ed:
        print(f"Error checking numeric values: {ed}")
        return False

def consistency_for_topsis(weights, impacts, dataset):
    try:
        return len(weights) == len(impacts) == len(dataset.columns) - 1
    except Exception as ed:
        print(f"Error checking consistency: {ed}")
        return False


def impacts(impacts):
    try:
        return all(impact in [-1, 1] for impact in impacts)
    except Exception as ed:
        print(f"Error checking impacts: {ed}")
        return False


def write_result_for_topsis(result_file_, results):
    df = pd.DataFrame({'Values': results})
    df['Rank'] = (len(df) + 1) - df['Values'].rank()
    df.to_csv(result_file_, index=False)
    print(f"Results have been saved to {result_file_}")


def topsis(data, weights, impacts, result_file_):
    dataset = read_data_for_topsis(dataset)
    if dataset is None:
        return

    weights_array_for_topsis, impacts_array_for_topsis= weights_and_impacts_for_topsis(weights,impacts)

    if not check_columns_for_topsis(dataset) or not check_num_values(dataset) or not consistency_for_topsis(weights_array_for_topsis, impacts_array_for_topsis, data) or not impacts(impacts_array_for_topsis):
        print("Input conditions are not Exiting.")
        return

    normalized_data = normalize_data_for_topsis(dataset)
    results = do_topsis(normalized_data, weights_array_for_topsis, impacts_array_for_topsis)
    write_result_for_topsis(result_file_, results)


if __name__ == "__main__":
    if len(sys.argv) != 5:
        print(
            "Usage: python topsis.py <InputDataFile> <Weights> <Impacts> <ResultFileName>")
    else:
        input_file = sys.argv[1]
        weights = sys.argv[2]
        impacts = sys.argv[3]
        result_file_ = sys.argv[4]

        topsis(data, weights, impacts, result_file_)



Usage: python topsis.py <InputDataFile> <Weights> <Impacts> <ResultFileName>
